**Importing the Librabries**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

**Reading the Dataset**

*The first five values and the last five values*

In [2]:
df = pd.read_csv("/content/large_sales_data_with_region.csv.crdownload")
df.head()

,date,product_id,product_name,quantity_sold,price_per_unit,total_sale_amount,region
0,2022-01-01,151,Product E,11.0,70.39,774.29,Lagos
1,2022-01-01,192,Product D,38.0,47.40,1801.20,Lagos
2,2022-01-01,114,Product B,NaN,50.72,NaN,Abuja
3,2022-01-01,171,Product D,20.0,12.62,252.40,Benin City
4,2022-01-01,160,Product B,20.0,9.23,184.60,Abuja


In [3]:
df.tail()

,date,product_id,product_name,quantity_sold,price_per_unit,total_sale_amount,region
2337820,2086-01-03,134,Product B,10.0,11.50,115.00,Abuja
2337821,2086-01-03,115,Product C,29.0,19.48,564.92,Abuja
2337822,2086-01-03,154,Product E,15.0,56.78,851.70,Abuja
2337823,2086-01-03,140,Product B,14.0,57.39,803.46,Port Harcourt
2337824,2086-01-03,198,Product D,46.0,78.61,3616.06,L


**checking the features and objects**

In [4]:
df.shape

(2337825, 7)

**Looking at the information of the whole dataset**

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2337825 entries, 0 to 2337824
Data columns (total 7 columns):
 #   Column             Dtype  
---  ------             -----  
 0   date               object 
 1   product_id         int64  
 2   product_name       object 
 3   quantity_sold      float64
 4   price_per_unit     float64
 5   total_sale_amount  float64
 6   region             object 
dtypes: float64(3), int64(1), object(3)
memory usage: 124.9+ MB


**checking for null values**

In [6]:
df.isnull().sum()

date                     0
product_id               0
product_name             0
quantity_sold        23372
price_per_unit       23494
total_sale_amount    46608
region                   0
dtype: int64

*The dataset shows a significant number of missing values*

1. **Handling The missing values in the dataset.**


* Quantity_sold: This column has 23,372 missing values. Given that this is a numerical column with significant missing data, I'll use median imputation i.e replacing the missing data with substituted values to fill in the missing values. The median is robust to outliers and provides a good estimate when data is missing.


* Price_per_unit: This column has 23,494 missing values. Similar to quantity_sold, I'll use median imputation for consistency.


* Total_sale_amount: This column has 46,608 missing values. To fill these values, I'll calculate total_sale_amount based on the formula: quantity_sold multiply by (*) price_per_unit. This approach ensures consistency with the available data.

*Quantity_sold Impute with Median*

In [7]:
quantity_sold_median = df['quantity_sold'].median()
quantity_sold_median

25.0

In [8]:
df['quantity_sold'].fillna(quantity_sold_median, inplace=True)

In [9]:
df["quantity_sold"].isnull().sum()

0

*Price_per_unit Impute with Median*

In [10]:
price_per_unit_median = df['price_per_unit'].median()
price_per_unit_median

52.53

In [11]:
df['price_per_unit'].fillna(price_per_unit_median, inplace=True)

In [12]:
df["price_per_unit"].isnull().sum()

0

In [13]:
# Calculate missing total_sale_amount
df['total_sale_amount'].fillna(df['quantity_sold'] * df['price_per_unit'], inplace=True)


**Justification**

* Median imputation is chosen for quantity_sold and price_per_unit due to its robustness against outliers and the large number of missing values.

* Calculation from existing data for total_sale_amount maintains consistency and leverages the available information.



2. **Standardize the Format of the Date Column**


In [14]:
#Checking the datatype

dtype = df["date"].dtype
print(dtype)

object


In [15]:
# Inspect unique values to understand the format
df['date'].unique()
print(df['date'])

0          2022-01-01
1          2022-01-01
2          2022-01-01
3          2022-01-01
4          2022-01-01
              ...    
2337820    2086-01-03
2337821    2086-01-03
2337822    2086-01-03
2337823    2086-01-03
2337824    2086-01-03
Name: date, Length: 2337825, dtype: object


In [16]:
# Convert the date column to datetime format
df['date'] = pd.to_datetime(df['date'], errors='coerce', format='%Y-%m-%d')

**Justification**

* *pd.to_datetime*,  ensures the dates are in a consistent datetime format for easier manipulation and analysis. The errors='coerce' parameter will handle any invalid date formats by converting them to NaT (Not a Time), which can then be addressed separately if necessary.

3. **Verify the Accuracy of the Date Column**

*Verification of Dates:*

Checking for any invalid or future dates:

In [17]:
# Check for any invalid dates or future dates
invalid_dates = df[df['date'] > pd.Timestamp.now()]
print(invalid_dates)


              date  product_id product_name  quantity_sold  price_per_unit  \
93500   2024-07-24         148    Product D            2.0           52.53   
93501   2024-07-24         122    Product C           13.0           72.27   
93502   2024-07-24         167    Product E           35.0           21.42   
93503   2024-07-24         134    Product B           35.0           39.44   
93504   2024-07-24         113    Product E            4.0           76.08   
...            ...         ...          ...            ...             ...   
2337820 2086-01-03         134    Product B           10.0           11.50   
2337821 2086-01-03         115    Product C           29.0           19.48   
2337822 2086-01-03         154    Product E           15.0           56.78   
2337823 2086-01-03         140    Product B           14.0           57.39   
2337824 2086-01-03         198    Product D           46.0           78.61   

         total_sale_amount         region  
93500              

**Steps to Handle Invalid or Future Dates**

1. *Identify and Inspect the Invalid Dates:*

I’ve already identified rows with future dates. To understand their context and review the rows.

2.  *Decide on an Action:*

Remove Future Dates: The future dates are not expected, removing these rows is the appropriate thing to do.

3. *Implement the Chosen Action:*

Remove Future Dates:

In [18]:
# Remove rows with future dates
df = df[df['date'] <= pd.Timestamp.now()]


In [19]:
# Check again for any future dates
future_dates_after_correction = df[df['date'] > pd.Timestamp.now()]
print(future_dates_after_correction)


Empty DataFrame
Columns: [date, product_id, product_name, quantity_sold, price_per_unit, total_sale_amount, region]
Index: []


In [20]:
# Inspect unique values to understand the format
df['date'].unique()
print(df['date'])


0       2022-01-01
1       2022-01-01
2       2022-01-01
3       2022-01-01
4       2022-01-01
           ...    
93495   2024-07-23
93496   2024-07-23
93497   2024-07-23
93498   2024-07-23
93499   2024-07-23
Name: date, Length: 93500, dtype: datetime64[ns]


In [21]:
df['date'].isnull().sum()

0

4. **Identify and Address Outliers**

*Detecting Outliers:*

Statistical methods like Z-scores is to be used for detecting outliers in numerical columns:

In [22]:
from scipy import stats

# Calculate Z-scores for numerical columns
z_scores = stats.zscore(df[['quantity_sold', 'price_per_unit', 'total_sale_amount']])
z_scores


,quantity_sold,price_per_unit,total_sale_amount
0,-0.996182,0.661202,-0.496836
1,0.920472,-0.181033,0.456930
2,-0.002361,-0.059406,-0.038291
3,-0.357297,-1.455194,-0.981553
4,-0.357297,-1.579386,-1.044524
...,...,...,...
93495,-0.002361,-1.183730,-0.750892
93496,1.346395,0.258951,1.211874
93497,-0.712233,0.219386,-0.403345
93498,-0.570259,1.534211,0.271677


In [23]:
# Identify outliers (e.g., Z-score > 3 or < -3)
outliers = (abs(z_scores) > 3).any(axis=1)
print(outliers)


0        False
1        False
2        False
3        False
4        False
         ...  
93495    False
93496    False
93497    False
93498    False
93499    False
Length: 93500, dtype: bool


**Justification**

* Z-scores are used to identify extreme values that deviate significantly from the mean. Removing outliers helps ensure that analysis results are not skewed by extreme values.


In [24]:
# Handling outliers - for simplicity, and removing them
df_cleaned = df[~outliers]
df_cleaned

,date,product_id,product_name,quantity_sold,price_per_unit,total_sale_amount,region
0,2022-01-01,151,Product E,11.0,70.39,774.29,Lagos
1,2022-01-01,192,Product D,38.0,47.40,1801.20,Lagos
2,2022-01-01,114,Product B,25.0,50.72,1268.00,Abuja
3,2022-01-01,171,Product D,20.0,12.62,252.40,Benin City
4,2022-01-01,160,Product B,20.0,9.23,184.60,Abuja
...,...,...,...,...,...,...,...
93495,2024-07-23,134,Product B,25.0,20.03,500.75,Abuja
93496,2024-07-23,160,Product C,44.0,59.41,2614.04,Abuja
93497,2024-07-23,159,Product C,15.0,58.33,874.95,Benin City
93498,2024-07-23,197,Product C,17.0,94.22,1601.74,Benin City


In [25]:
#checking for the shape of the dataset
df_cleaned.shape

(93182, 7)

In [26]:
#Checking for duplicate values
df_cleaned.duplicated().sum()

0

In [27]:
#checking for null values
df_cleaned.isnull().sum()

date                 0
product_id           0
product_name         0
quantity_sold        0
price_per_unit       0
total_sale_amount    0
region               0
dtype: int64

In [28]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 93182 entries, 0 to 93499
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               93182 non-null  datetime64[ns]
 1   product_id         93182 non-null  int64         
 2   product_name       93182 non-null  object        
 3   quantity_sold      93182 non-null  float64       
 4   price_per_unit     93182 non-null  float64       
 5   total_sale_amount  93182 non-null  float64       
 6   region             93182 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 5.7+ MB


In [29]:
# Save the cleaned DataFrame to a CSV file
sales_data = df_cleaned.to_csv('sales_data.csv', index=False)
sales_data

In [30]:
df = pd.read_csv("/content/sales_data.csv")
df.head()

,date,product_id,product_name,quantity_sold,price_per_unit,total_sale_amount,region
0,2022-01-01,151,Product E,11.0,70.39,774.29,Lagos
1,2022-01-01,192,Product D,38.0,47.40,1801.20,Lagos
2,2022-01-01,114,Product B,25.0,50.72,1268.00,Abuja
3,2022-01-01,171,Product D,20.0,12.62,252.40,Benin City
4,2022-01-01,160,Product B,20.0,9.23,184.60,Abuja


In [31]:
df.columns

Index(['date', 'product_id', 'product_name', 'quantity_sold', 'price_per_unit',
       'total_sale_amount', 'region'],
      dtype='object')